In [81]:
import random
import numpy as np
import cec2017.functions as functions
import copy

In [82]:
Dim = 30
NP = 100
bound = 100

In [83]:
X = np.random.uniform(low=-bound, high=bound, size=Dim)
X_min = X.min()
X_max = X.max()
X_list = np.empty([NP, Dim])
for i in range(NP):
    for j in range(Dim):
        X_list[i][j] = X_min + random.uniform(0, 1) * (X_max - X_min)

X_list

array([[-35.54321502,   1.11808898,  13.15900065, ...,  63.3312991 ,
        -28.65594184, -11.57223154],
       [-48.38246557, -75.21427591,  74.62734459, ...,  74.16102899,
        -65.76173362,  49.92100365],
       [-65.81269737,  42.50763421,  72.82172921, ...,  15.06945483,
        -26.67896521,  44.75117879],
       ...,
       [ 90.07155656,  52.87171607,  42.47114436, ..., -45.16575696,
         35.8333904 ,  79.37212529],
       [ 64.78390671, -27.0912336 , -28.61543085, ..., -49.78214807,
         72.74739115,  64.31402017],
       [-87.46367516,  -3.22310038,  23.9262236 , ...,  37.87494378,
         35.41253609, -21.49064606]])

In [84]:
""":parameter for DE"""
T_DE = 100
no = 5
CR = 0.9  # rate of crossover

""":parameter for MMS"""
mms_e = 0.000001
mms_F = 0.5

In [85]:
""":parameter k and F: for current/rand/1"""
k = 0.3
F = 0.5

In [86]:
""":parameter for crossoverSBX"""
sbx_n = 10

In [87]:
# f = functions.f1

In [88]:
def feas(x):
    if x.max() <= bound and x.min() >= -bound:
        return 1
    else:
        return 0

In [89]:
def select_better_fit(x1, x2, f):
    if f(x1) <= f(x2):
        return x1
    else:
        return x2

In [90]:
def current_to_rand(x_list, r1, r2, r3, i):
    return np.add(x_list[i], k * np.subtract(x_list[r1], x_list[i]), F * np.subtract(x_list[r2], x_list[r3]))

In [91]:
def immsade(xx_list, r1, r2, r3):
    immsade_omega = random.uniform(0.7, 1.0)
    immsade_F = random.uniform(0.5, 0.8)
    v_delta = np.add(np.multiply(immsade_omega, xx_list[r1]),
                     immsade_F * np.subtract(xx_list[r2], xx_list[r3]))
    v_delta = np.maximum(v_delta, -100)
    v_delta = np.minimum(v_delta, 100)
    return v_delta

In [92]:
def crossover_SBX(x_i, v_j, f):
    beta = 0
    while isinstance(beta, complex):
        sbx_rand = random.random()
        if sbx_rand <= 0.5:
            beta = (2 * sbx_rand) ** (1 / (1 + sbx_n))
        else:
            beta = (1 / (2 - sbx_n * 2)) ** (1 / (1 + sbx_n))

    c1 = 0.5 * np.add((1 + beta) * x_i, (1 - beta) * v_j)  # crossover
    c2 = 0.5 * np.add((1 - beta) * x_i, (1 + beta) * v_j)
    return select_better_fit(c1, c2, f)

In [93]:
def DE(f):
    x_list = copy.deepcopy(X_list)
    x_best = x_list[0]
    # print(x_list[0])

    t = 1
    while t <= T_DE:
        for i in range(NP):
            u_best = np.empty([Dim])
            for j in range(Dim):
                r_list = random.sample(range(NP), 5)
                for m in range(no):
                    while r_list[m] == i:
                        r_list[m] = random.randint(0, Dim - 1)

                v_j = np.empty([Dim])
                v_j = immsade(x_list, r_list[1], r_list[2], r_list[3])

                cr_rand = random.random()
                if cr_rand <= CR:
                    u_j = crossover_SBX(x_list[i], v_j, f)
                else:
                    u_j = x_list[i]

                if j == 0:
                    u_best = u_j
                else:
                    u_best = select_better_fit(u_best, u_j, f)

                if f(u_best) <= f(x_list[i]) and feas(u_best) == 1:
                    x_list[i] = u_best

                if f(x_list[i]) < f(x_best) and feas(x_list[i]) == 1:
                    x_best = x_list[i]

        t += 1
    return x_best


In [94]:
""":parameter: for AGDE"""
T_AGDE = 100

In [95]:
def index_of_X_best(xx_list, f):
    index = 0
    min_fit = f(xx_list[0])
    for i in range(len(xx_list)):
        if f(xx_list[i]) < min_fit:
            index = i
            min_fit = f(xx_list[i])
    return index


def index_of_X_worst(xx_list, f):
    index = 0
    max_fit = f(xx_list[0])
    for i in range(len(xx_list)):
        if f(xx_list[i]) > max_fit:
            index = i
            max_fit = f(xx_list[i])
    return index


def compare_max(delta):
    """
    lower bound
    :param delta: delta
    :return: turn into lower bound delta
    """

    em = np.linspace(-100, -100, Dim)
    delta = np.maximum(delta, em)
    return delta


def compare_min(delta):
    on = np.linspace(100, 100, Dim)
    delta = np.minimum(delta, on)
    # for i in range(len(delta)):
    #     delta[i] = min(delta[i], 1)
    return delta

def agde(xx_list, agde_F, agde_p, f):
    agde_NP = int(NP * agde_p)
    r_list = random.randint(agde_NP * 2 - 1, NP - 1)
    x_p_worst = xx_list[index_of_X_worst(xx_list[agde_NP: agde_NP * 2 - 1], f)]
    x_p_best = xx_list[index_of_X_best(xx_list[: agde_NP - 1], f)]
    v_delta = np.add(xx_list[r_list], np.multiply(agde_F, np.subtract(x_p_best, x_p_worst)))
    return v_delta

In [96]:
def AGDE(f):
    xx_list = copy.deepcopy(X_list)
    x_best = xx_list[0]
    # print(xx_list[0])

    G = 0
    p_j = 0.5
    s_j_sum = 0
    while G <= T_AGDE:  # iteration of DE
        n_s_j = 0
        f_j = 1
        for i in range(NP):  # iteration of generation
            agde_F = random.uniform(0.1, 1)
            Cr = 0
            if G == 1:
                if random.uniform(0, 1) <= 0.5:
                    Cr = random.uniform(0.05, 0.15)
                else:
                    Cr = random.uniform(0.9, 1)
            else:
                if random.uniform(0, 1) <= p_j:
                    Cr = random.uniform(0.05, 0.15)
                else:
                    Cr = random.uniform(0.9, 1)
            j_random = random.randint(1, NP)

            u_best = np.empty([Dim])
            for j in range(Dim):
                vj = agde(xx_list, agde_F, p_j, f)
                vj = compare_max(vj)
                vj = compare_min(vj)
                if random.uniform(0, 1) < Cr or j == j_random:
                    u_j = crossover_SBX(xx_list[i], vj, f)
                else:
                    u_j = xx_list[i]

                if j == 0:
                    u_best = u_j
                else:
                    u_best = select_better_fit(u_best, u_j, f)

                if f(u_best) <= f(xx_list[i]) and feas(u_best) == 1:
                    xx_list[i] = u_best

                if f(xx_list[i]) < f(x_best) and feas(xx_list[i]) == 1:
                    x_best = xx_list[i]

        G += 1

        s_j = n_s_j / (n_s_j + f_j) + 0.01
        s_j_sum += s_j
        p_s_j = s_j / s_j_sum
        if (((G - 1) * p_j + p_s_j) / G) < 0.5:
            p_j = ((G - 1) * p_j + p_s_j) / G

    return x_best

In [97]:
""":parameter: for GSK"""
T_GSK = 100
GSK_p = 0.1
k_f = 0.01   # 0.5
k_r = 0.1   # 0.9
K = 10

In [98]:
def junior(xx_list, r, i, f):
    if random.uniform(0, 1) <= k_r:
        if f(xx_list[i]) > f(xx_list[r]):
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(xx_list[max(0, i - 1)], xx_list[min(NP - 1, i + 1)]),
                                          np.subtract(xx_list[r], xx_list[i])))
        else:
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(xx_list[max(0, i - 1)], xx_list[min(NP - 1, i + 1)]),
                                          np.subtract(xx_list[i], xx_list[r])))
    else:
        v_delta = xx_list[i]
    return v_delta


def senior(xx_list, i, D_junior, f):
    if random.uniform(0, 1) <= k_r:
        D_senior = NP - D_junior
        NP_1 = int(D_senior * GSK_p)
        r_list = random.randint(D_junior - 1 + NP_1 * 2, NP - 1)
        x_p_worst = xx_list[index_of_X_worst(xx_list[D_junior - 1 + NP_1: D_junior - 1 + NP_1 * 2 - 1], f)]
        x_p_best = xx_list[index_of_X_best(xx_list[max(D_junior - 1, 0): D_junior - 1 + NP_1 - 1], f)]
        if f(xx_list[i]) > f(xx_list[r_list]):
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(x_p_best, x_p_worst),
                                          np.subtract(xx_list[r_list], xx_list[i])))
        else:
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(x_p_best, x_p_worst),
                                          np.subtract(xx_list[i], xx_list[r_list])))
    else:
        v_delta = xx_list[i]
    return v_delta

In [99]:
def GSK(f):
    """
    DE algorithm
    :param x_list:
    :return: best x by DE
    """
    xxx_list = copy.deepcopy(X_list)

    x_best = xxx_list[0]
    # print(xxx_list[0])
    G = 0

    while G <= T_GSK:  # iteration of DE
        D_junior = int(NP * pow((1 - G / T_GSK), K))

        for i in range(NP):  # iteration of generation

            u_best = np.empty([Dim])
            for j in range(Dim):
                vj = np.empty([Dim])
                r = random.randint(0, NP - 1)
                if j <= D_junior:
                    vj = junior(xxx_list, r, j, f)
                else:
                    vj = senior(xxx_list, j, D_junior, f)

                vj = compare_max(vj)
                vj = compare_min(vj)

                if j == 0:
                    u_best = vj
                else:
                    u_best = select_better_fit(u_best, vj, f)

                if f(u_best) <= f(xxx_list[i]) and feas(u_best) == 1:
                    xxx_list[i] = u_best

                if f(xxx_list[i]) < f(x_best) and feas(xxx_list[i]) == 1:
                    x_best = xxx_list[i]

        G += 1

    return x_best


In [ ]:
import os
i = 0
test_list = np.empty([len(functions.all_functions), 3])
test_path = r'D:\profile\codes\python_projects\ocp\test\output\D30_cpu'
for i in range(51):
    for f in functions.all_functions:
        x_best_DE_fit = 0
        x_best_AGDE_fit = 0
        x_best_GSK_fit = 0
        x_best_DE_fit = f(DE(f))
        x_best_AGDE_fit = f(AGDE(f))
        x_best_GSK_fit = f(GSK(f))
        print(f.__name__, x_best_DE_fit, x_best_AGDE_fit, x_best_GSK_fit, "\n")
        test_list[i][0] = x_best_DE_fit
        test_list[i][1] = x_best_AGDE_fit
        test_list[i][2] = x_best_GSK_fit
        i += 1
    np.savetxt(os.path.join(test_path, str(i) + ".txt"), test_list)




f1 109435434299.20612 531640029811.3256 1349715541503.2996 

f2 1.349911553687143e+31 9.855765313602567e+43 2.2399441390760688e+46 

f3 43155.97937580376 59230.18523479779 218973.3748128256 

f4 1737.2590443524841 6975.91124240426 16881.272409884947 

f5 667.4662938130336 945.1664078599492 1176.1054496312815 

f6 661.0034778506084 703.6071856853891 766.7838398551588 

f7 1004.4907474697447 2065.590562118915 3249.68353498548 

f8 994.0249235070938 1090.1527963545586 1254.906636854534 

f9 3366.22865584563 7921.469797334585 34161.14702622843 

f10 6130.697833290114 7558.940824803381 10045.896960631111 

f11 5313.366573086167 8283.469842312956 39164.44571363688 

f12 6053753532.03842 55312562226.75439 167285313980.8411 

f13 1922020497.3755467 128789295021.31244 255162558923.79865 

f14 698025.2507922259 104990.38119745757 21449611.13316221 

f15 23818845.69869695 4750354820.417643 53223610066.28072 

f16 3023.7988787234103 4982.075226032086 8205.702903072399 



c:\users\chris\appdata\local\programs\python\python39\lib\site-packages\cec2017\basic.py:227: RuntimeWarning: invalid value encountered in true_divide
  t = np.abs(tjx - np.round(tjx)) / tj


f17 nan nan nan 

f18 225421.12517844583 17169120.649721943 160679350.5454473 

f19 23912370.445084326 3550660224.8850775 52204580143.53042 

f20 nan nan nan 

f21 2460.81115470609 2612.815325170474 2895.1127372372475 



In [ ]:
import cec2017.utils as utils
for f in functions.all_functions:
    utils.surface_plot(f, points=120, dimension=Dim)